$\large\textbf{Lab 7. } $


$\large\textbf{Exercise 2 : Scalability}$

In [17]:
import numpy as np
import matplotlib.pyplot as plt
import math
import timeit


In [18]:
np.random.seed(1000)

In [19]:
 def evalf(x,lam,A):  
  assert type(x) is np.ndarray   
  fval = np.linalg.norm(np.matmul(A,x) - y)
  fval = 0.5 * (fval)**2+(lam/2)*np.matmul(x.T,x)
  return (fval)


In [20]:
def evalg(x,lam,A):
  assert type(x) is np.ndarray 
  grad=(lam*x+np.matmul(A.T, np.matmul(A, x) - y))
  return grad

In [21]:
def evalh(x,lam,A):
  assert type(x) is np.ndarray  
  return np.matmul(A.T,A)+lam*np.identity(d)

In [22]:
def compute_B_k(s, y, B_k):
  I=np.identity(ds[i])
  assert type(s) is np.ndarray 
  assert type(y) is np.ndarray 
  assert type(B_k) is np.ndarray
  mu = 1 / np.dot(np.squeeze(np.asarray(y)), np.squeeze(np.asarray(s)))
  a1 = np.outer(np.multiply(mu, s), np.transpose(y))
  a2 = np.outer(np.multiply(mu, y), np.transpose(s))
  a = np.subtract(I, a1)
  b = np.subtract(I, a2)
  c = np.outer(np.multiply(mu, s), np.transpose(s))
  d = np.matmul(np.matmul(a, B_k), b)
  z = np.add(d, c)
  return z

In [23]:
#line search type 
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1

In [24]:
def compute_D_k(x):
  assert type(x) is np.ndarray
  mat = evalh(x,A)
  return np.linalg.inv(mat)

In [25]:
def compute_steplength_backtracking_scaled_direction(x, lam,gradf, alpha_start, rho, gamma, B_k): 
  assert type(x) is np.ndarray
  assert type(gradf) is np.ndarray
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  alpha = alpha_start
  p = - gradf
  D_k = B_k
  
  while evalf(x + alpha*np.matmul(D_k,p),lam,A) > evalf(x,lam,A) + gamma * alpha* (np.matmul(np.matrix.transpose(gradf), np.matmul(D_k,p)) ):
    alpha = alpha * rho
  return alpha 

In [26]:
def find_minimizer_Newtonmethod(start_x,A,lam, tol, line_search_type, *args):
  assert type(start_x) is np.ndarray  
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg(x,lam,A)
  h_x = evalh(x,lam,A)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  x_array_newton = []
  f_array_newton = []
  while (np.linalg.norm(g_x) > tol): 
    D_k = np.linalg.inv(evalh(x,lam,A))
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x,lam, g_x, alpha_start, rho, gamma, D_k)
    elif line_search_type == CONSTANT_STEP_LENGTH: 
      step_length = 1.0
    else:
      raise ValueError('Line search type unknown. Please check!')  
    x = np.subtract(x, np.multiply(step_length,np.matmul(D_k, g_x))) 
    k = k + 1
    g_x = evalg(x,lam,A) 
  return x,  k, x_array_newton, f_array_newton

In [27]:
def find_minimizer_BFGS_scaling(start_x,A, lam,tol, line_search_type, *args):
  assert type(start_x) is np.ndarray 
  assert type(tol) is float and tol>=0 
  
  x = start_x
  g_x = evalg(x,lam,A)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])

  k = 0
  B_k = np.identity(d)
  x_old = x
  x_array_bfgs = []
  f_array_bfgs = []
  while (np.linalg.norm(g_x) > tol): 
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_scaled_direction(x_old,lam, g_x, alpha_start, rho, gamma, B_k)
    elif line_search_type == CONSTANT_STEP_LENGTH: 
      step_length = 1.
    else:
      raise ValueError('Line search type unknown. Please check!')
    x_new = np.subtract(x_old, np.multiply(step_length, np.matmul(B_k,g_x))) 
    g_x = evalg(x_new,lam,A)
    s = np.subtract(x_new, x_old)
    y = np.subtract(evalg(x_new,lam,A), evalg(x_old,lam,A)) 
    B_k = compute_B_k(s, y, B_k)
    x_old = x_new
    k = k + 1
  return x_new, k, x_array_bfgs, f_array_bfgs

In [ ]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5
N=200
ds=[1000,5000,10000,20000,25000,50000,100000,200000,500000,1000000]
lam=0.1

k1=[]
eps = np.random.randn(N,1)
for i in range(np.size(ds)):
  d=ds[i]
  A=np.random.randn(N,d)
  x=np.ones((d,1))
  y=np.dot(A,x)+eps
  start=timeit.default_timer()
  x, k, x_array_newton, f_array_newton = find_minimizer_Newtonmethod(x.reshape((d,1)),A,lam, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
  end=timeit.default_timer()
  k1.append(end-start)
  print('The total CPU time taken for column value',ds[i],'is',end-start)
  print('The minimizer for column value',ds[i],'is',np.array(x))
  print('L2 for column value',ds[i],'is',x-np.ones((d,1)))
print(k1)

The total CPU time taken for column value 1000 is 1.1534832780000102
The minimizer for column value 1000 is [[ 4.31206549e-01]
 [ 8.96688377e-01]
 [-5.91573164e-01]
 [-3.74167894e-02]
 [-3.62351351e-01]
 [-2.71692466e-01]
 [-1.36583487e-01]
 [ 2.45635727e-01]
 [ 3.39961498e-01]
 [ 5.90967534e-01]
 [-1.74923148e-02]
 [-1.81082861e-01]
 [ 8.79513354e-02]
 [ 8.48368459e-01]
 [ 1.21994727e-01]
 [ 1.02170402e+00]
 [ 7.42114488e-01]
 [-3.49114982e-01]
 [ 3.30659764e-02]
 [ 4.85655115e-01]
 [ 6.37551475e-01]
 [ 4.29520105e-01]
 [ 1.89662386e-01]
 [ 8.86773181e-01]
 [ 1.08943445e-02]
 [ 6.53082750e-01]
 [-4.36350776e-01]
 [ 7.86301955e-01]
 [-3.99520078e-01]
 [ 1.04286816e+00]
 [ 2.02910125e-01]
 [-5.54856490e-02]
 [-7.93388685e-03]
 [ 2.51681222e-01]
 [ 6.50449653e-01]
 [ 6.69979943e-01]
 [-8.42115400e-01]
 [ 1.53046860e-01]
 [ 6.66809675e-01]
 [ 7.03241672e-02]
 [-3.24713153e-01]
 [-3.68681012e-01]
 [ 1.95668135e-01]
 [ 1.19668124e-01]
 [-7.97731286e-01]
 [ 1.09958891e+00]
 [-3.96219063e-04]

$\large{comment:}$

The failure for newton's method occurs at the dimension 20000 .

In [ ]:
alpha = 0.9
rho = 0.5
gamma = 0.5
my_tol= 1e-5
N=200
ds=[1000,5000,10000,20000,25000,50000,100000,200000,500000,1000000]
lam=0.1
k1=[]
eps = np.random.randn(N,1)
for i in range(np.size(ds)):
  d=ds[i]
  A=np.random.randn(N,d)
  x=np.ones((d,1))
  y=np.dot(A,x)+eps
  start=timeit.default_timer()
  x, k, x_array_newton, f_array_newton = find_minimizer_BFGS_scaling(x.reshape((d,1)),A,lam, my_tol, BACKTRACKING_LINE_SEARCH, alpha, rho, gamma)
  end=timeit.default_timer()
  k1.append(end-start)
  print('The CPU time taken for column value',ds[i],'is',end-start)
  print('The minimizer for column value',ds[i],'is',np.array(x))
  print('L2 for column value',ds[i],'is',x-np.ones((d,1)))
print(k1)

The CPU time taken for column value 1000 is 14.43289334699989
The minimizer for column value 1000 is [[ 2.18757255e-01]
 [ 3.65526283e-01]
 [ 1.33741977e-01]
 [ 3.66163732e-01]
 [ 8.06422821e-01]
 [-3.68947752e-01]
 [ 3.43227468e-01]
 [-3.69895387e-01]
 [-1.44144554e-01]
 [ 3.16895074e-01]
 [ 5.00422471e-01]
 [-9.63109593e-02]
 [-5.11124090e-01]
 [-7.27423714e-01]
 [ 3.33749706e-01]
 [ 1.50832326e-01]
 [-3.08331652e-02]
 [-7.63909147e-02]
 [ 1.40760979e+00]
 [-4.37280235e-01]
 [ 8.65109745e-01]
 [ 1.69965444e-01]
 [ 7.53918042e-01]
 [-4.08948273e-01]
 [-1.20814266e-01]
 [-6.30125602e-01]
 [ 2.82935924e-01]
 [-1.14878608e-01]
 [ 3.41112389e-01]
 [ 7.86245946e-01]
 [-3.52899491e-01]
 [-5.86765743e-01]
 [ 3.75365422e-01]
 [-4.20869864e-01]
 [-3.20819739e-01]
 [-1.65829615e-01]
 [ 2.35758309e-01]
 [-3.97001050e-01]
 [ 1.54903356e-01]
 [ 2.46819032e-01]
 [-2.03363506e-01]
 [-3.62740462e-01]
 [ 5.53800440e-02]
 [ 3.65184482e-01]
 [ 5.45527271e-01]
 [ 6.70933797e-01]
 [ 2.47277181e-01]
 [-3.0

Here The system ran for more than 30 minutes,Hence the failure occur  for column no. 5000. 